In [71]:
import os
import pandas as pd

In [72]:
path_wd = "/Users/wenphan/Documents/GitHub_ddl/domino-gfp-ml"
path_artifacts = os.path.join(path_wd, "artifacts")
path_adult = os.path.join(path_wd, "data", "adult.csv")

In [73]:
pd_adult = pd.read_csv(path_adult, header=None, index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'gender',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'income'])

In [74]:
pd_adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [75]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [76]:
ct = ColumnTransformer(
    [("scaling", StandardScaler(), ['age', 'hours-per-week']),
     ("onehot", OneHotEncoder(sparse=False),
     ['workclass', 'education', 'gender', 'occupation'])])

In [77]:
pd_features = pd_adult.drop("income", axis=1)
pd_target = pd_adult[["income"]]

In [78]:
from sklearn.model_selection import train_test_split

In [79]:
pd_adult_X_train, pd_adult_X_test, pd_adult_y_train, pd_adult_y_test = train_test_split(
    pd_features, pd_target, random_state=0)

In [80]:
ct.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('scaling',
                                 StandardScaler(copy=True, with_mean=True,
                                                with_std=True),
                                 ['age', 'hours-per-week']),
                                ('onehot',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 ['workclass', 'education', 'gender',
                                  'occupation'])],
                  verbose=False)

In [81]:
X_train.shape

(24420, 14)

In [82]:
import pickle

In [83]:
with open(os.path.join(path_artifacts, "coltransformer.pkl"), "wb") as f:
    pickle.dump(ct, f)
    f.close()

In [84]:
with open(os.path.join(path_artifacts, "coltransformer.pkl"), "rb") as f:
    cx = pickle.load(f)
    f.close()

In [85]:
X_train_trans = cx.transform(X_train)

In [86]:
from sklearn.linear_model import LogisticRegression

In [87]:
model_logreg = LogisticRegression()

In [88]:
model_logreg.fit(X_train_trans, y_train)

/Users/wenphan/py_envs/jira_zendesk/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [89]:
X_test_trans = ct.transform(X_test)

In [90]:
model_logreg.score(X_test_trans, y_test)

0.8092371944478566